In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

from CRUD import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "accuser"
password = "notMyPassword"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))

app = JupyterDash('SimpleExample')

PAGE_SIZE = 10
                               
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#<------ Layout ------>
                               
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'height':'10%', 'width':'10%'})),
    html.Center(html.H1(children="Jonathan Handy")),        
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'WR'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'MWR'},
            {'label': 'Disaster Rescue or Individual Tracking', 'value': 'DRIT'},
            {'label': 'Reset', 'value': 'RE'}
        ],
        labelStyle={'display': 'inline-block'}
    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        
        page_current=0,
        page_size=PAGE_SIZE,
        page_action='custom',
        

        sort_action='custom',
        sort_mode='multi',
        sort_by=[]
        
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#<------ Pagination w/ Filtering & Multi-Column Sorting ------>
@app.callback(
    Output('datatable-id', 'data'),
    [Input('datatable-id', "page_current"),
    Input('datatable-id', "page_size"),
    Input('datatable-id', 'sort_by'),
    Input('filter-type', 'value')])
def update_table(page_current, page_size, sort_by, filter_type):
    dff = df
    
    if filter_type == 'WR':
        dff = pd.DataFrame.from_records(shelter.read(
            {'$and':[
                {'$or':[
                    {'breed':{
                        '$in':[
                            'Labrador Retriever Mix',
                            'Chesapeake Bay Retriever',
                            'Newfoundland'
                        ]
                    }}
                ]},
                {'sex_upon_outcome': 'Intact Female'},
                {'$and': [
                    {'age_upon_outcome_in_weeks':{'$gte':26}},
                    {'age_upon_outcome_in_weeks':{'$lte':156}}
                ]}
            ]}
        ))
    elif filter_type == 'MWR':
        dff = pd.DataFrame.from_records(shelter.read(
            {'$and':[
                {'$or':[
                    {'breed':{
                        '$in':[
                            'German Shepherd', 
                            'Alaskan Malamute', 
                            'Old English Sheepdog', 
                            'Siberian Husky',
                            'Rottweiler'
                        ]
                    }}
                ]},
                {'sex_upon_outcome': 'Intact Male'},
                {'$and': [
                    {'age_upon_outcome_in_weeks':{'$gte':26}},
                    {'age_upon_outcome_in_weeks':{'$lte':156}}
                ]}
            ]}
        ))
    elif filter_type == 'DRIT':
        dff = pd.DataFrame.from_records(shelter.read(
            {'$and':[
                {'$or':[
                    {'breed':{
                        '$in':[
                            'Doberman Pinscher', 
                            'German Shepherd', 
                            'Golden Retriever', 
                            'Bloodhound', 
                            'Rottweiler'
                        ]
                    }}
                ]},
                {'sex_upon_outcome': 'Intact Male'},
                {'$and': [
                    {'age_upon_outcome_in_weeks':{'$gte':20}},
                    {'age_upon_outcome_in_weeks':{'$lte':300}}
                ]}
            ]}
        ))
    elif filter_type == 'RE':
        dff = pd.DataFrame.from_records(shelter.read({}))


    if len(sort_by):
        dff = dff.sort_values(
            [col['column_id'] for col in sort_by],
            ascending=[
                col['direction'] == 'asc'
                for col in sort_by
            ],
            inplace=False
        )

    page = page_current
    size = page_size
    return dff.iloc[page * size: (page + 1) * size].to_dict('records')


# <------ Data Table Styling ------>
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    breedCounts = dff['breed'].value_counts()
    breeds = dff['breed'].unique()
    return [
        dcc.Graph(            
            figure = px.bar(dff, x=breeds, y=breedCounts, title='Available Breeds'),
        )    
    ]

#<------ Geolocation Map ------>
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]



app